# Bibliotecas

In [ ]:
# ! pip install xgboost
# ! pip install scikit-optimize

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import (train_test_split,
                                     KFold,
                                     cross_val_score,
                                     RandomizedSearchCV)
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Dataset

## Treino

In [5]:
treino = pd.read_csv('/content/drive/MyDrive/Datasets/Titanic/train.csv')

treino

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
treino.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

'Embarked' é variável categórica -> os valores NA serão subtituídos pela moda

'Age' é variável contínua -> os valores NA serão subtituídos pela média

In [7]:
treino['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
moda_embarked = treino['Embarked'].mode()[0]

treino.loc[treino['Embarked'].isnull(), 'Embarked'] = moda_embarked

treino['Embarked'].isnull().sum()

0

In [9]:
media_age = treino['Age'].mean()

treino.loc[treino['Age'].isnull(), 'Age'] = media_age

treino['Age'].isnull().sum()

0

In [10]:
X_treino = treino.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis='columns') # Exclui features não interessantes

X_treino['Sex'] = X_treino['Sex'].map({'male': 0, 'female': 1}) # Sex variável discreta

X_treino['Embarked'] = X_treino['Embarked'].map({'C': 0, 'S': 1, 'Q': 2}) # Embarked variável discreta


y_treino = treino['Survived']

## Teste

In [11]:
teste = pd.read_csv('/content/drive/MyDrive/Datasets/Titanic/test.csv')

teste

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [12]:
teste.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [13]:
teste.loc[teste['Age'].isnull(), 'Age'] = media_age

media_fare = treino['Fare'].mean()

teste.loc[teste['Fare'].isnull(), 'Fare'] = media_fare

teste.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

O teste é como se fosse a produção, onde os dados não são conhecidos

Por essa razão os valores NA do teste foram substituídos pela moda Embarked e média Age do treino (dados conhecidos)

In [14]:
X_teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis='columns') # Exclui features não interessantes

X_teste['Sex'] = X_teste['Sex'].map({'male': 0, 'female': 1}) # Sex variável discreta

X_teste['Embarked'] = X_teste['Embarked'].map({'C': 0, 'S': 1, 'Q': 2}) # Embarked variável discreta

In [15]:
X_teste.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

# Modelo

In [16]:
cv = KFold(n_splits=10, random_state=1, shuffle=True) # Cross validation

clf = RandomForestClassifier(random_state=31) # Classificador

media_scores_sem_tunning = cross_val_score(clf, X_treino, y_treino, scoring='accuracy', cv=cv, n_jobs=-1)

## Tunning randomizado

In [17]:
param_space = {'bootstrap': [True],
               'max_depth': [6, 8, 10, 12, 14],
               'max_features': ['auto', 'sqrt', 'log2'],
               'min_samples_leaf': [2, 3, 4],
               'min_samples_split': [2, 3, 4, 5],
               'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

forest_rand_search = RandomizedSearchCV(clf,
                                        param_space,
                                        n_iter=32,
                                        scoring='accuracy',
                                        cv=5,
                                        n_jobs=-1)

forest_rand_search.fit(X_treino, y_treino)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=31),
                   n_iter=32, n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [6, 8, 10, 12, 14],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [2, 3, 4],
                                        'min_samples_split': [2, 3, 4, 5],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   scoring='accuracy')

In [18]:
forest_rand_search.best_params_

{'n_estimators': 200,
 'min_samples_split': 3,
 'min_samples_leaf': 4,
 'max_features': 'log2',
 'max_depth': 14,
 'bootstrap': True}

In [19]:
forest_rand_search.best_estimator_

RandomForestClassifier(max_depth=14, max_features='log2', min_samples_leaf=4,
                       min_samples_split=3, n_estimators=200, random_state=31)

## Tunning bayesiano

In [20]:
search_space = {'bootstrap': Categorical([True, False]),
                'max_depth': Integer(6, 20),
                'max_features': Categorical(['auto', 'sqrt', 'log2']),
                'min_samples_leaf': Integer(2, 10),
                'min_samples_split': Integer(2, 10),
                'n_estimators': Integer(100, 500)}

forest_bayes_search = BayesSearchCV(clf,
                                    search_space,
                                    n_iter=32,
                                    scoring='accuracy',
                                    n_jobs=-1,
                                    cv=5)

forest_bayes_search.fit(X_treino, y_treino)

BayesSearchCV(cv=5, estimator=RandomForestClassifier(random_state=31),
              n_iter=32, n_jobs=-1, scoring='accuracy',
              search_spaces={'bootstrap': Categorical(categories=(True, False), prior=None),
                             'max_depth': Integer(low=6, high=20, prior='uniform', transform='normalize'),
                             'max_features': Categorical(categories=('auto', 'sqrt', 'log2'), prior=None),
                             'min_samples_leaf': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=10, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=500, prior='uniform', transform='normalize')})

In [21]:
forest_bayes_search.best_params_

OrderedDict([('bootstrap', True),
             ('max_depth', 20),
             ('max_features', 'log2'),
             ('min_samples_leaf', 4),
             ('min_samples_split', 5),
             ('n_estimators', 100)])

In [22]:
forest_bayes_search.best_estimator_

RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=4,
                       min_samples_split=5, random_state=31)

## XGBoost

In [23]:
clf = xgb.XGBClassifier()

# Cross validation apenas para comparação entre modelos
cv_xgboost_sem_tunning = cross_val_score(clf, X_treino, y_treino, scoring='accuracy')

## Tunnin XGBoost -> booster dart 

In [24]:
clf = xgb.XGBClassifier(max_depth=2,
                        booster='dart')

cv_xgboost_dart = cross_val_score(clf, X_treino, y_treino, scoring='accuracy')

## Tunnin XGBoost -> booster gbtree 

In [25]:
clf = xgb.XGBClassifier(max_depth=2,
                        booster='gbtree')

cv_xgboost_gbtree = cross_val_score(clf, X_treino, y_treino, scoring='accuracy')

## Tunnin XGBoost -> booster gblinear 

In [26]:
clf = xgb.XGBClassifier(max_depth=2,
                        booster='gblinear')

cv_xgboost_gblinear = cross_val_score(clf, X_treino, y_treino, scoring='accuracy')

# Comparativo de acurácia

In [27]:
print('\n \
Random Forest sem tunning--------{:.2f} \n \
Random Forest randomizado--------{:.2f} \n \
Random Forest bayesiano----------{:.2f} \n \
XGBoost--------------------------{:.2f} \n \
XGBoost dart---------------------{:.2f} \n \
XGBoost gbtree-------------------{:.2f} \n \
XGBoost gblinear-----------------{:.2f}'.format(media_scores_sem_tunning.mean(),
                                                forest_rand_search.best_score_,
                                                forest_bayes_search.best_score_,
                                                cv_xgboost_sem_tunning.mean(),
                                                cv_xgboost_dart.mean(),
                                                cv_xgboost_gbtree.mean(),
                                                cv_xgboost_gblinear.mean()))


 Random Forest sem tunning--------0.82 
 Random Forest randomizado--------0.83 
 Random Forest bayesiano----------0.83 
 XGBoost--------------------------0.78 
 XGBoost dart---------------------0.79 
 XGBoost gbtree-------------------0.79 
 XGBoost gblinear-----------------0.68


# Aplicação do melhor modelo

## Cross validation score

In [28]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# Random Forest bayesiano
clf_melhor_modelo = RandomForestClassifier(max_depth=20,
                                           max_features='log2',
                                           min_samples_leaf=2,
                                           min_samples_split=10,
                                           random_state=31)

In [29]:
scores_melhor_modelo = cross_val_score(clf_melhor_modelo, X_treino, y_treino, scoring='accuracy', cv=cv, n_jobs=-1)

media_scores_melhor_modelo = scores_melhor_modelo.mean()

media_scores_melhor_modelo

0.8272284644194757

## Predição de valores

In [30]:
clf_melhor_modelo.fit(X_treino, y_treino)

y_pred = clf_melhor_modelo.predict(X_teste)

In [31]:
y_pred.shape

(418,)

In [32]:
scores_melhor_modelo_pred = cross_val_score(clf_melhor_modelo, X_teste, y_pred, scoring='accuracy', cv=cv, n_jobs=-1)

media_scores_melhor_modelo_pred = scores_melhor_modelo_pred.mean()

media_scores_melhor_modelo_pred

0.9306620209059234